In [6]:
# Define Hurricane damage file

hurricane_name = 'All' # match the names in summary_cbsa_damage_all_2014-2024.csv
hurricane_aggregated_damage_file = f'./climada_pyrecodes_examples/climada_inputs/summary_cbsa_damage_all_2014-2024.csv'

In [7]:
# Get the damage results for the hurricane
import csv

with open(hurricane_aggregated_damage_file, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    all_hurricane_damage = [row for row in reader]

# hurricane_damage = [row for row in all_hurricane_damage if hurricane_name in row['event']]
hurricane_damage = all_hurricane_damage


In [8]:
# Get the permit data for the affected CBSAs per unit - TODO: Replace with actual data retrieval logic
import random

cbsa_names = [row['cbsa_name'] for row in hurricane_damage]
cbsa_permit_data = {}
unit_columns = ['1 Unit', '2 Units', '3 and 4 Units', '5 Units or More']
for cbsa_name in cbsa_names:
    cbsa_permit_data[cbsa_name] = {unit: random.randint(10, 100) for unit in unit_columns}

In [11]:
# Define pyrecodes light - very simple recovery simulator
import csv

def get_reconstruction_capacity(building_permits_per_cbsa: dict) -> dict:
    reconstruction_capacity = int(building_permits_per_cbsa['1 Unit']) + int(building_permits_per_cbsa['2 Units']) * 2 + int(building_permits_per_cbsa['3 and 4 Units']) * 3 + int(building_permits_per_cbsa['5 Units or More']) * 5
    return reconstruction_capacity

def get_housing_unit_month_demand(row: dict, repair_time: dict) -> int:
    housing_unit_month_demand = 0
    for damage_state in repair_time.keys():
        housing_unit_month_demand += float(row[damage_state]) * repair_time[damage_state]
    return housing_unit_month_demand

def get_recovery_time(damage_summary: dict, reconstruction_capacity: dict, repair_time: dict) -> int:
    recovery_time = {}
    for row in damage_summary:
        county_name = row['cbsa_name']
        housing_unit_month_demand = get_housing_unit_month_demand(row, repair_time)
        county_reconstruction_capacity = reconstruction_capacity.get(county_name, None)
        minimal_recovery_time = get_minimal_recovery_time(row, repair_time)
        if county_reconstruction_capacity is None:
            print(f"County {county_name} not found in reconstruction capacity data.")
        elif county_reconstruction_capacity == 0:
            print(f"County {county_name} has zero reconstruction capacity.")
        else:
            recovery_time[county_name] = max(minimal_recovery_time, int(housing_unit_month_demand / county_reconstruction_capacity))
    return recovery_time

def get_minimal_recovery_time(row: dict, repair_time: dict) -> int:
    """
    Get minimal recovery time for a county based on the damage states.
    If there is a damaged building, return the repair time for that damage state.
    """
    minimal_recovery_time = 0
    for damage_state in repair_time.keys():
        if float(row[damage_state]) > 0:
            minimal_recovery_time = max(repair_time[damage_state], minimal_recovery_time)
    return minimal_recovery_time

def get_output_dict(damage_summary: dict, reconstruction_capacity: dict, recovery_time: dict, hurricane_name: str) -> list:
    output_dict = []
    for row in damage_summary:
        county_name = row['cbsa_name']
        row['reconstruction_capacity'] = reconstruction_capacity.get(county_name, None)
        row['recovery_potential'] = recovery_time.get(county_name, None)
        output_dict.append({
            'event': row['event'],
            'cbsa_name': row['cbsa_name'],
            'reconstruction_capacity': reconstruction_capacity.get(county_name, None),
            'recovery_potential [months]': recovery_time.get(county_name, None)
        })
    return output_dict

reconstruction_capacity = {}
for cbsa_name, permit_data in cbsa_permit_data.items():
    # print(f"CBSA: {cbsa_name}, Permits: {permit_data}")
    reconstruction_capacity[cbsa_name] = get_reconstruction_capacity(permit_data)

repair_time = {'DS0_units': 0, 'DS1_units': 1, 'DS2_units': 1, 'DS3_units': 3, 'DS4_units': 8} # in months HAZUS except DS4 which is 8 months, not 6 months as in HAZUS

recovery_time = get_recovery_time(hurricane_damage, reconstruction_capacity, repair_time)
output_dict = get_output_dict(hurricane_damage, reconstruction_capacity, recovery_time, hurricane_name)

# Specify the CSV file name
filename = f'recovery_potential_{hurricane_name}.csv'

# Get the headers from the keys of the first dict
fieldnames = output_dict[0].keys()

# Write to CSV
with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_dict)

print(recovery_time)

{'Deltona-Daytona Beach-Ormond Beach, FL': 13, 'Orlando-Kissimmee-Sanford, FL': 52, 'Palm Bay-Melbourne-Titusville, FL': 26, 'Port St. Lucie, FL': 5, 'Sebastian-Vero Beach-West Vero Corridor, FL': 3, 'Atlantic City-Hammonton, NJ': 1, 'Cambridge, MD': 0, 'Dover, DE': 0, 'Elizabeth City, NC': 0, 'Greenville, NC': 0, 'Jacksonville, NC': 27, 'Kill Devil Hills, NC': 0, 'Morehead City, NC': 38, 'Murrells Inlet, SC': 0, 'Myrtle Beach-Conway-North Myrtle Beach, SC': 5, 'New Bern, NC': 11, 'New York-Newark-Jersey City, NY-NJ': 0, 'Ocean Pines, MD': 1, 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD': 0, 'Salisbury, MD': 1, 'Seaford, DE': 3, 'Vineland, NJ': 0, 'Virginia Beach-Chesapeake-Norfolk, VA-NC': 0, 'Washington, NC': 1, 'Wilmington, NC': 111, 'Non-CBSA 37': 23, 'Non-CBSA 51': 1, 'Brunswick-St. Simons, GA': 1, 'Douglas, GA': 6, 'Gainesville, FL': 1, 'Hilton Head Island-Bluffton-Port Royal, SC': 1, 'Hinesville, GA': 0, 'Jesup, GA': 1, 'Lake City, FL': 0, 'Savannah, GA': 1, 'Tallahassee, FL': 5